<a href="https://colab.research.google.com/github/Ayush086/Generative-AI/blob/main/Day12-Retrievers/langchain_retrievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Wikipedia Retriever

In [2]:
from langchain_community.retrievers import WikipediaRetriever

In [3]:
# Initialize the retriever
retriever = WikipediaRetriever(top_k_results = 3, lang='en')

In [4]:
# user query
query = "the geopolitical history of india and pakistan from the perspective of china"

# get relevant docs
docs = retriever.invoke(query)

In [6]:
# check retrieved content
for i, doc in enumerate(docs):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"Content:\n {doc.page_content}...")
  print()

 ---------- Result 1  ----------
Content:
 The Indo-Pakistani war of 1971, also known as the third India-Pakistan war, was a military confrontation between India and Pakistan that occurred during the Bangladesh Liberation War in East Pakistan from 3 December 1971 until the Pakistani capitulation in Dhaka on 16 December 1971.  The war began with Pakistan's Operation Chengiz Khan, consisting of preemptive aerial strikes on eight Indian air stations. The strikes led to India declaring war on Pakistan, marking their entry into the war for East Pakistan's independence, on the side of Bengali nationalist forces. India's entry expanded the existing conflict with Indian and Pakistani forces engaging on both the eastern and western fronts. Thirteen days after the war started, India achieved a clear upper hand, and the Eastern Command of the Pakistan military signed the instrument of surrender on 16 December 1971 in Dhaka, marking the formation of East Pakistan as the new nation of Bangladesh. A

# Vector Store Retriever

In [9]:
import os

os.environ['COHERE_API_KEY'] = "43w9iLmkVlcDMzLiYXfPfnRgdRtG7NZDT6wCSzcK"

In [10]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.schema import Document

In [11]:
# s1: source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [13]:
# s2: initialize embedding model
cohere_embeddings = CohereEmbeddings(user_agent="my-app")

# s3: chroma vector store + adding documents
vector_store = Chroma.from_documents(
        documents = documents,
         embedding=cohere_embeddings,
         persist_directory="chroma_db",
         collection_name='retriever_store'
     )

In [19]:
# s4: convert vectorstore into a retriver
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [20]:
# testing
query = "What is Chroma used for ?"
results = retriever.invoke(query)

In [21]:
for i, doc in enumerate(results):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"{doc.page_content}")
  print()

 ---------- Result 1  ----------
Chroma is a vector database optimized for LLM-based search.

 ---------- Result 2  ----------
OpenAI provides powerful embedding models.



In [22]:
answer = vector_store.similarity_search(query, k = 2)

In [23]:
for i, doc in enumerate(answer):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"{doc.page_content}")
  print()

 ---------- Result 1  ----------
Chroma is a vector database optimized for LLM-based search.

 ---------- Result 2  ----------
OpenAI provides powerful embedding models.



**Might be thinking that we can do same thing with vectorstore.smilarity_search then why to use retrievers ?**

.similarity_search() uses only one kind on searching strategy whereas retrievers have various types of searching algos which we can try and in many cases those algos outperformed the general algo in .similarity_search()

# MMR

In [24]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [25]:
from langchain_community.vectorstores import FAISS

In [26]:
# init embeddings model
cohere_embeddings = CohereEmbeddings(user_agent="my-app")

# create vector store
vector_store = FAISS.from_documents(
    documents = docs,
    embedding = cohere_embeddings
)

In [39]:
from re import search
# enable MMR in retriever
retriever = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {
        "k": 3,
        "lambda_mult": 0.6  # relevance-diversity balance
    }
)

In [40]:
query = "Tell me about langchain"
result = retriever.invoke(query)

In [41]:
for i, doc in enumerate(result):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"{doc.page_content}")
  print()

 ---------- Result 1  ----------
LangChain is used to build LLM based applications.

 ---------- Result 2  ----------
LangChain supports Chroma, FAISS, Pinecone, and more.

 ---------- Result 3  ----------
Embeddings are vector representations of text.



# Multiquery Retriever

In [43]:
from langchain_cohere import ChatCohere
from langchain.retrievers.multi_query import MultiQueryRetriever

In [44]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [45]:
# init embeddings
cohere_embeddings = CohereEmbeddings(user_agent="my-app")

In [46]:
# FAISS vector store
vector_store = FAISS.from_documents(
    documents = all_docs,
    embedding = cohere_embeddings
)

In [49]:
# create retrievers
# normal retriever
similarity_retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 5})

# multi-query retriever
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = vector_store.as_retriever(search_kwargs={'k': 5}),
    llm = ChatCohere(model='command-r')
)

In [50]:
query = "how to improve energy level and maintain balance ?"

In [51]:
normal_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [52]:
print("Normal Similarity Search Results: \n")
for i, doc in enumerate(normal_results):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"{doc.page_content}")
  print()

Normal Similarity Search Results: 

 ---------- Result 1  ----------
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 ---------- Result 2  ----------
Consuming leafy greens and fruits helps detox the body and improve longevity.

 ---------- Result 3  ----------
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 ---------- Result 4  ----------
Deep sleep is crucial for cellular repair and emotional regulation.

 ---------- Result 5  ----------
Regular walking boosts heart health and can reduce symptoms of depression.



In [53]:
print("Multiquery Retriver: \n")
for i, doc in enumerate(multiquery_results):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"{doc.page_content}")
  print()

Multiquery Retriver: 

 ---------- Result 1  ----------
The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.

 ---------- Result 2  ----------
Python balances readability with power, making it a popular system design language.

 ---------- Result 3  ----------
Black holes bend spacetime and store immense gravitational energy.

 ---------- Result 4  ----------
The solar energy system in modern homes helps balance electricity demand.

 ---------- Result 5  ----------
Consuming leafy greens and fruits helps detox the body and improve longevity.

 ---------- Result 6  ----------
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 ---------- Result 7  ----------
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 ---------- Result 8  ----------
Deep sleep is crucial for cellular repair and emotional regulation.

 ---------- Result 9  ----------
Regular walking boosts heart health and can reduce sympto

# Contextual Compression Retriever

In [54]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [55]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [56]:
# init embeddings
cohere_embeddings = CohereEmbeddings(user_agent="my-app")

# FAISS vector store
vector_store = FAISS.from_documents(
    documents = docs,
    embedding = cohere_embeddings
)


In [57]:
base_retriever = vector_store.as_retriever(search_kwargs={'k': 5})

In [58]:
# compressor using LLM
llm = ChatCohere(model='command-r')
compressor = LLMChainExtractor.from_llm(llm)

In [59]:
# contextual compressor retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [60]:
query = "what is photosynthesis ?"
compressed_results = compression_retriever.invoke(query)

In [61]:
for i, doc in enumerate(compressed_results):
  print(f" {'-'*10} Result {i+1}  {'-'*10}")
  print(f"{doc.page_content}")
  print()

 ---------- Result 1  ----------
Photosynthesis is the process by which green plants convert sunlight into energy.

 ---------- Result 2  ----------
The chlorophyll in plant cells captures sunlight during photosynthesis.

 ---------- Result 3  ----------
Photosynthesis does not occur in animal cells.

